# Task 4 extra: Sequential Pattern Mining

In [145]:
import copy
import pandas as pd
from spmf import Spmf
from operator import neg
from itertools import groupby
from collections import defaultdict

In [69]:
def spmf_encode(dataset):
    """
    e.g.
    
    dataset = [
        # sequence: list of events
        [['a'], ['a', 'b', 'c'], ['a', 'c'], ['c']],  # event: [list of strings]
        [['a'], ['c'], ['b', 'c']], 
        [['a', 'b'], ['d'], ['c'], ['b'], ['c']], 
        [['a'], ['c'], ['b'], ['c']]
    ]
    
    
    @CONVERTED_FROM_TEXT
    @ITEM=0=a
    @ITEM=1=b
    @ITEM=2=c
    @ITEM=3=d
    0 -1 0 1 2 -1 0 2 -1 2 -1 -2
    0 -1 2 -1 1 2 -1 -2
    0 1 -1 3 -1 2 -1 1 -1 2 -1 -2
    0 -1 2 -1 1 -1 2 -1 -2
    """
    
    items = sorted(set(item.replace(' ', '_') for sequence in dataset 
                       for event in sequence 
                       for item in event))

    labels_dict = dict()

    spmf_str = '@CONVERTED_FROM_TEXT' + '\n'
    for idx, item in enumerate(items):
        spmf_str += '@ITEM=' + str(idx) + '=' + item + '\n'
        labels_dict[item] = idx

    for sequence in dataset:
        for event in sequence:
            for item in event:
                spmf_str += str(labels_dict[item.replace(' ', '_')]) + ' '
            spmf_str += '-1' + ' '
        spmf_str += '-2' + '\n'
    return spmf_str

In [70]:
def spmf_sequence_length(spmf_sequence):
    """
    Computes the length of a SPMF sequence (sum of the length of the contained itemsets)
    """
    return sum(len(i.split()) for i in spmf_sequence)

In [71]:
spmf_sequence_length(['a', 'b c', 'a', 'b c d'])

7

## PrefixSpan: Sequential Pattern Mining by Pattern-Growth

In [119]:
dataset = [
    # sequence: list of events
    [['a'], ['a', 'b', 'c'], ['a', 'c'], ['c']],  # event: [list of strings]
    [['a'], ['c'], ['b', 'c']], 
    [['a', 'b'], ['d'], ['c'], ['b'], ['c']], 
    [['a'], ['c'], ['b'], ['c']]
]

In [120]:
def is_subsequence(main_sequence, subsequence):
    """
    Recursive method that checks if `subsequence` is a subsequence of `main_sequence`
    """

    def is_subsequence_recursive(subsequence_clone, start=0):
        """
        Function for the recursive call of is_subsequence
        """
        # check if empty: end of recursion, all itemsets have been found
        if not subsequence_clone:
            return True
        # retrieves element of the subsequence and removes is from subsequence 
        first_elem = set(subsequence_clone.pop(0))
        # search for the first itemset...
        for i in range(start, len(main_sequence)):
            if set(main_sequence[i]).issuperset(first_elem):
                # and recurse
                return is_subsequence_recursive(subsequence_clone, i + 1)
        return False

    return is_subsequence_recursive(subsequence.copy()) # start recursion

In [121]:
sequence = [['a'], ['b', 'c'], ['d'], ['a', 'e']]

In [122]:
is_subsequence(sequence, [['a'], ['b', 'c'], ['e']])

True

In [123]:
is_subsequence(sequence, [['a'], ['b', 'd']])

False

In [124]:
def project_sequence(sequence, prefix, new_event):
    """
    Projects a sequence according to a given prefix

    Args:
        sequence: the sequence the projection is built from
        prefix: the prefix that is searched for in the sequence
        new_event: if set to True, the first itemset is ignored
    Returns:
        If the sequence does not contain the prefix, then None.
        Otherwise, a new sequence starting from the position of the prefix, including the itemset that includes the prefix
    """
    result = None
    for i, itemset in enumerate(sequence):
        if result is None:
            if not new_event or i > 0:
                if all(x in itemset for x in prefix):
                    result = [list(itemset)]
        else:
            result.append(copy.copy(itemset))
    return result

In [125]:
sequence = [['a'], ['b', 'c'], ['a', 'c'], ['c']]
project_sequence(sequence, ['b'], False)

[['b', 'c'], ['a', 'c'], ['c']]

In [126]:
project_sequence(sequence, ['a', 'c'], False)

[['a', 'c'], ['c']]

In [127]:
project_sequence(sequence, ['a'], False)

[['a'], ['b', 'c'], ['a', 'c'], ['c']]

In [128]:
project_sequence(sequence, ['a'], True)

[['a', 'c'], ['c']]

In [129]:
def project_dataset(dataset, prefix, new_event):
    """
    Projects a dataset according to a given prefix

    Args:
        dataset: the dataset the projection is built from
        prefix: the prefix that is searched for in the sequence
        new_event: if set to True, the first itemset is ignored
    Returns:
        A (potentially empty) list of sequences
    """
    projected_db = []
    for sequence in dataset:
        seq_proj = project_sequence(sequence, prefix, new_event)
        if not seq_proj is None:
            projected_db.append(seq_proj)
    return projected_db

In [130]:
project_dataset(dataset, ['c'], False)

[[['a', 'b', 'c'], ['a', 'c'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b'], ['c']]]

In [52]:
def sequence_length(sequence):
    """
    Computes the length of the sequence (sum of the length of the contained itemsets)
    """
    return sum(len(i) for i in sequence)

In [132]:
sequence_length([['a'], ['b', 'c'], ['a'], ['b', 'c', 'd']])

7

In [133]:
def generate_items(dataset):
    """
    Generates a list of all items that are contained in a dataset
    """
    return sorted(set([item for sequence in dataset for event in sequence for item in even]))

def generate_item_supports(dataset, ignore_first_event=False, prefix=[]):
    """
    Computes a defaultdict that maps each item in the dataset to its support
    """
    result = defaultdict(int)
    for sequence in dataset:
        if ignore_first_event:
            sequence = sequence[1:]
        cooccurring_items = set()
        for itemset in sequence:
            if all(x in itemset for x in prefix):
                for item in itemset:
                    if not item in prefix:
                        cooccurring_items.add(item)
        for item in cooccurring_items:
            result[item] += 1
    return sorted(result.items())

In [134]:
def prefix_span(dataset, min_sup):
    """
    The PrefixSpan algorithm. Computes the frequent sequences in a seqeunce dataset.

    Args:
        dataset: a list of sequences, for which the frequent (sub-)sequences are computed
        min_sup: the minimum support that makes a sequence frequent
    Returns:
        A list of tuples (s, c), where s is a frequent sequence, and c is the count for that sequence
    """
    
    def prefix_span__recursive(dataset, min_sup, prev_prefixes=[]):
        result = []

        # add a new item to the last element (==same time)
        item_count_same_event = generate_item_supports(dataset, False, prefix=prev_prefixes[-1])
        for item, count in item_count_same_event:
            if count >= min_sup and item > prev_prefixes[-1][-1]:
                new_prefix = copy.deepcopy(prev_prefixes)
                new_prefix[-1].append(item)
                result.append((new_prefix, count))
                result.extend(prefix_span__recursive(project_dataset(dataset, new_prefix[-1], False), min_sup, new_prefix))

        # add a new event to the prefix
        item_count_subsequent_events = generate_item_supports(dataset, True)
        for item, count in item_count_subsequent_events:
            if count >= min_sup:
                new_prefix = copy.deepcopy(prev_prefixes)
                new_prefix.append([item])
                result.append((new_prefix, count))
                result.extend(prefix_span__recursive(project_dataset(dataset, [item], True), min_sup, new_prefix))
        return result
    
    result = []
    item_counts = generate_item_supports(dataset)
    min_sup *= len(dataset)
    for item, count in item_counts:
        if count >= min_sup:
            new_prefix = [[item]]
            result.append((new_prefix, count))
            result.extend(prefix_span__recursive(project_dataset(dataset, [item], False), min_sup, new_prefix))
    result.sort(key=lambda tup: (tup[1], neg(sequence_length(tup[0]))), reverse=True)
    return result

In [135]:
prefix_span(dataset, min_sup=0.5)

[([['a']], 4),
 ([['b']], 4),
 ([['c']], 4),
 ([['a'], ['b']], 4),
 ([['a'], ['c']], 4),
 ([['c'], ['c']], 4),
 ([['a'], ['c'], ['c']], 4),
 ([['b'], ['c']], 3),
 ([['c'], ['b']], 3),
 ([['a'], ['b'], ['c']], 3),
 ([['a'], ['c'], ['b']], 3),
 ([['a', 'b']], 2),
 ([['b', 'c']], 2),
 ([['a', 'b'], ['c']], 2),
 ([['a'], ['b', 'c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2),
 ([['a', 'b'], ['c'], ['c']], 2),
 ([['a'], ['c'], ['b'], ['c']], 2)]

In [136]:
spmf_dataset = spmf_encode(dataset)
print(spmf_dataset)

@CONVERTED_FROM_TEXT
@ITEM=0=a
@ITEM=1=b
@ITEM=2=c
@ITEM=3=d
0 -1 0 1 2 -1 0 2 -1 2 -1 -2
0 -1 2 -1 1 2 -1 -2
0 1 -1 3 -1 2 -1 1 -1 2 -1 -2
0 -1 2 -1 1 -1 2 -1 -2



In [78]:
spmf = Spmf('PrefixSpan_AGP', input_direct=spmf_dataset, arguments=[0.5])  # min_sup
spmf.run()
freq_patterns = spmf.to_pandas_dataframe()
freq_patterns = list(zip(freq_patterns.pattern, freq_patterns.sup))
freq_patterns.sort(key=lambda tup: (tup[1], neg(spmf_sequence_length(tup[0]))), reverse=True)
freq_patterns

>/home/donato/donato.meoli.95@gmail.com/DataMiningUniPi/DM_Group18_TASK4/spmf.jar
=============  Algorithm - STATISTICS =============
 Total time ~ 7 ms
 Frequent sequences count : 19
 Max memory (mb):12.550086975097656
Content at file spmf-output.txt.tmp

Post-processing to show result in terms of string values.
Post-processing completed.



[(['a'], 4),
 (['b'], 4),
 (['c'], 4),
 (['a', 'b'], 4),
 (['a', 'c'], 4),
 (['c', 'c'], 4),
 (['a', 'c', 'c'], 4),
 (['b', 'c'], 3),
 (['c', 'b'], 3),
 (['a', 'b', 'c'], 3),
 (['a', 'c', 'b'], 3),
 (['a b'], 2),
 (['b c'], 2),
 (['b', 'c', 'c'], 2),
 (['c', 'b', 'c'], 2),
 (['a', 'b c'], 2),
 (['a', 'c', 'b', 'c'], 2),
 (['a b', 'c'], 2),
 (['a b', 'c', 'c'], 2)]

## Loading the new Customer Supermarket dataset

In [73]:
df = pd.read_csv('../dataset/new_customer_supermarket.csv', sep='\t', index_col=0)
df

,BasketID,BasketDate,Sale,CustomerID,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313.0,22386,JUMBO BAG PINK POLKADOT,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313.0,21499,BLUE POLKADOT WRAP,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313.0,21498,RED RETROSPOT WRAP,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313.0,22379,RECYCLING BAG RETROSPOT,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313.0,20718,RED RETROSPOT SHOPPER BAG,10,12.50
...,...,...,...,...,...,...,...,...
363571,581587,2011-09-12 12:50:00,0.85,12680.0,22613,PACK OF SPACEBOY NAPKINS,12,10.20
363572,581587,2011-09-12 12:50:00,2.10,12680.0,22899,CHILDRENS APRON DOLLY GIRL,6,12.60
363573,581587,2011-09-12 12:50:00,4.15,12680.0,23254,CHILDRENS CUTLERY DOLLY GIRL,4,16.60
363574,581587,2011-09-12 12:50:00,4.15,12680.0,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,16.60


In [74]:
df.dtypes

BasketID        int64
BasketDate     object
Sale          float64
CustomerID    float64
ProdID         object
ProdDescr      object
Qta             int64
TotSale       float64
dtype: object

In [75]:
df = df.astype({'BasketDate': 'datetime64',
                'BasketID': 'object',
                'CustomerID': 'object'})

## Data Modeling

In [76]:
df.sort_values('BasketDate', inplace=True)
df['BasketDayOfYear'] = df['BasketDate'].dt.dayofyear
df

,BasketID,BasketDate,Sale,CustomerID,ProdID,ProdDescr,Qta,TotSale,BasketDayOfYear
20767,542776,2011-01-02 08:23:00,0.30,15240.0,17021,NAMASTE SWAGAT INCENSE,36,10.80,2
20769,542776,2011-01-02 08:23:00,4.95,15240.0,21485,RETROSPOT HEART HOT WATER BOTTLE,3,14.85,2
20768,542776,2011-01-02 08:23:00,3.75,15240.0,21218,RED SPOTTY BISCUIT TIN,5,18.75,2
20766,542776,2011-01-02 08:23:00,2.95,15240.0,22083,PAPER CHAIN KIT RETROSPOT,12,35.40,2
20765,542776,2011-01-02 08:23:00,4.65,15240.0,22835,HOT WATER BOTTLE I AM SO POORLY,4,18.60,2
...,...,...,...,...,...,...,...,...,...
254281,570876,2011-12-10 17:19:00,3.75,16085.0,23394,POSTE FRANCE CUSHION COVER,1,3.75,344
254280,570876,2011-12-10 17:19:00,1.55,16085.0,46000M,POLYESTER FILLER PAD,1,1.55,344
254279,570876,2011-12-10 17:19:00,1.45,16085.0,46000S,POLYESTER FILLER PAD,1,1.45,344
254291,570876,2011-12-10 17:19:00,1.65,16085.0,22469,HEART OF WICKER SMALL,3,4.95,344


In [49]:
baskets_sequences = df.groupby('CustomerID').apply(
    lambda customer: customer.groupby('BasketDayOfYear')['ProdDescr'].apply(
        lambda prod: list(k for k, v in groupby(prod))))  # removing consecutive duplicate products
baskets_sequences = baskets_sequences.groupby('CustomerID').apply(list).tolist()
baskets_sequences[:1]

[[['RED RETROSPOT OVEN GLOVE',
   'BLACK CANDELABRA TLIGHT HOLDER',
   'SET RED RETROSPOT TEA TOWELS',
   'AIRLINE BAG VINTAGE JET SET BROWN',
   'REGENCY CAKESTAND TIER',
   'CHOCOLATE CALCULATOR',
   'ALARM CLOCK BAKELIKE ORANGE',
   'ALARM CLOCK BAKELIKE GREEN',
   'DOG PICTURE PLAYING CARDS',
   'BOX OF ASSORTED COLOUR TEASPOONS',
   'SANDWICH BATH SPONGE',
   'PACK OF SPACEBOY CAKE CASES',
   'SET OF TINS VINTAGE BATHROOM',
   'PACK OF MUSHROOM CAKE CASES',
   'SMALL HEART MEASURING SPOONS',
   'TEA TIME OVEN GLOVE',
   'BLUE NEW BAROQUE CANDLESTICK CANDLE',
   'WOODLAND CHARLOTTE BAG',
   'RED RETROSPOT OVEN GLOVE DOUBLE',
   'AIRLINE BAG VINTAGE JET SET WHITE',
   'TOOTHPASTE TUBE PEN',
   'PINK NEW BAROQUECANDLESTICK CANDLE',
   'ALARM CLOCK BAKELIKE RED',
   'ALARM CLOCK BAKELIKE PINK',
   'RED TOADSTOOL LED NIGHT LIGHT',
   'SWEETHEART FAIRY CAKE CASES',
   'ALARM CLOCK BAKELIKE CHOCOLATE',
   'MINI LADLE LOVE HEART RED',
   'TEATIME FAIRY CAKE CASES'],
  ['SWEETHEART FAIRY C

PrefixSpan custom implementation:

In [50]:
%time freq_patterns = prefix_span(baskets_sequences, min_sup=0.05)
list(filter(lambda freq_pattern: sequence_length(freq_pattern[0]) >= 2, freq_patterns))

CPU times: user 11.8 s, sys: 0 ns, total: 11.8 s
Wall time: 11.8 s


[([['WHITE HANGING HEART TLIGHT HOLDER'],
   ['WHITE HANGING HEART TLIGHT HOLDER']],
  366),
 ([['GREEN REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER']],
  304),
 ([['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']], 301),
 ([['REGENCY CAKESTAND TIER'], ['REGENCY CAKESTAND TIER']], 295),
 ([['GREEN REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER']],
  282),
 ([['PAPER CHAIN KIT CHRISTMAS', 'PAPER CHAIN KIT VINTAGE CHRISTMAS']], 280),
 ([['ASSORTED COLOUR BIRD ORNAMENT'], ['ASSORTED COLOUR BIRD ORNAMENT']], 274),
 ([['PARTY BUNTING'], ['PARTY BUNTING']], 272),
 ([['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT']], 266),
 ([['PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER']],
  256),
 ([['GARDENERS KNEELING PAD CUP OF TEA', 'GARDENERS KNEELING PAD KEEP CALM']],
  254),
 ([['RED HANGING HEART TLIGHT HOLDER', 'WHITE HANGING HEART TLIGHT HOLDER']],
  254),
 ([['REGENCY CAKESTAND TIER', 'ROSES REGENCY TEACUP AND SAUCER']], 254),
 ([

PrefixSpan SPMF implementation:

In [72]:
spmf_baskets_sequences = spmf_encode(baskets_sequences)
spmf = Spmf('PrefixSpan', input_direct=spmf_baskets_sequences, arguments=[0.05])  # min_sup
spmf.run()
freq_patterns = spmf.to_pandas_dataframe()
freq_patterns = freq_patterns[freq_patterns['pattern'].map(spmf_sequence_length) >= 2]
freq_patterns = list(zip(freq_patterns.pattern, freq_patterns.sup))
freq_patterns.sort(key=lambda tup: (tup[1], neg(spmf_sequence_length(tup[0]))), reverse=True)
freq_patterns

>/home/donato/donato.meoli.95@gmail.com/DataMiningUniPi/DM_Group18_TASK4/spmf.jar
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 824 ms
 Frequent sequences count : 262
 Max memory (mb) : 258.9457778930664
 minsup = 211 sequences.
 Pattern count : 262

Post-processing to show result in terms of string values.
Post-processing completed.



[(['WHITE_HANGING_HEART_TLIGHT_HOLDER', 'WHITE_HANGING_HEART_TLIGHT_HOLDER'],
  366),
 (['JUMBO_BAG_RED_RETROSPOT', 'JUMBO_BAG_RED_RETROSPOT'], 301),
 (['REGENCY_CAKESTAND_TIER', 'REGENCY_CAKESTAND_TIER'], 295),
 (['ASSORTED_COLOUR_BIRD_ORNAMENT', 'ASSORTED_COLOUR_BIRD_ORNAMENT'], 274),
 (['PARTY_BUNTING', 'PARTY_BUNTING'], 272),
 (['LUNCH_BAG_RED_RETROSPOT', 'LUNCH_BAG_RED_RETROSPOT'], 266),
 (['LUNCH_BAG_BLACK_SKULL', 'LUNCH_BAG_BLACK_SKULL'], 220),
 (['LUNCH_BAG_SUKI_DESIGN', 'LUNCH_BAG_SUKI_DESIGN'], 217),
 (['SET_OF_CAKE_TINS_PANTRY_DESIGN', 'SET_OF_CAKE_TINS_PANTRY_DESIGN'], 215)]